In [16]:
import re

In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
import sys

sys.path.append('../../')

In [19]:
from src.indexing import get_multivector_retriever
from src.generation import QA_SYSTEM_PROMPT, QA_PROMPT, LLAMA_PROMPT_TEMPLATE, MIXTRAL_PROMPT_TEMPLATE
from src.generation import get_model, format_docs, get_rag_chain
from langchain_core.documents import Document

from src.ingestion import load_pdf

import os
import chromadb
import uuid

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
DATA_PATH = 'D:\Ahmed\saudi-rag-project\data'
RAW_DOCS_PATH = os.path.join(DATA_PATH, "raw")
CHROMA_PATH = os.path.join(DATA_PATH, "chroma")

MODEL_NAME = "meta-llama/Llama-3-8b-chat-hf"

In [ ]:
RETRIEVERS = [
    dict(collection_name="PARENT_1200_400_CHILD_300_0_infloat_multilingual_e5_small", embedding_model_name="intfloat/multilingual-e5-small"),
    dict(collection_name="PARENT_1200_400_QUESTIONS_COMBINED", embedding_model_name="text-embedding-3-small"),
    dict(collection_name="PARENT_1200_400_QUESTIONS_SPLIT", embedding_model_name="text-embedding-3-small"),
    dict(collection_name="PARENT_1200_400_SUMMARIES", embedding_model_name="text-embedding-3-small"),
]

### Load benchmark

In [13]:
benchmark = pd.read_csv("../../data/benchmark.csv")

In [15]:
def extract_numbers(text):
    text = re.sub(',', '', text)

    # This pattern matches both integers and decimal numbers
    pattern = r'\b\d+\.?\d*\b'

    # Find all matches in the text and return them as a list of floats or integers
    numbers = re.findall(pattern, text)

    # Convert the extracted number strings to appropriate float or int types
    return [float(num) if '.' in num else int(num) for num in numbers]

### Load different retrievers

In [ ]:
retrivers = dict()

for retriever_dict in RETRIEVERS

    retrivers[retrivers['collection_name']] = get_multivector_retriever(persistent_client, retrivers['embedding_model_name'], retrivers['collection_name'], DATA_PATH)

### Assess